In [1]:
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense
from keras.utils import np_utils
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import keras

In [2]:
fed=pd.read_csv("D:/DATA SCIENCES/AI Assignments/Deep Learning Issues_ ANN/AIModule7Assignmentfedex/fedex.csv")

In [3]:
fed=fed.dropna() # droppping na value as its a bigger dataset
fed.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Actual_Shipment_Time',
       'Planned_Shipment_Time', 'Planned_Delivery_Time', 'Carrier_Name',
       'Carrier_Num', 'Planned_TimeofTravel', 'Shipment_Delay', 'Source',
       'Destination', 'Distance', 'Delivery_Status'],
      dtype='object')

In [4]:
#checking  correlation

corr1=pd.DataFrame(fed.iloc[:,:-1].corr())

#here correlation of year is least removing year column and categorical feature as it cannot be consider as i need to check for selectkBEst

fed_new=fed.drop(['Year','Source','Destination','Carrier_Name'],axis=1)

In [5]:
#output is categorical in tw0 categories so we are checking best feature using Selecet Baise

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [6]:
# separating x and y

x=fed_new.iloc[:,0:11]
y=fed_new['Delivery_Status']

In [7]:
#taking absolute value as we cannot i/p the negative value for Select K best

x['Shipment_Delay']=x['Shipment_Delay'].abs()
x['Planned_Delivery_Time']=x['Planned_Delivery_Time'].abs()
x['Planned_TimeofTravel']=x['Planned_TimeofTravel'].abs()

In [8]:
# Rank fo all Features

ordered_rank_features=SelectKBest(score_func=chi2,k=11)
ordered_feature=ordered_rank_features.fit(x,y)
dfscores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])
dfcolumns=pd.DataFrame(x.columns)
features_rank=pd.concat([dfcolumns,dfscores],axis=1)    
features_rank.columns=['Features','Score']
features_rank

,Features,Score
0,Month,1.373603e+03
1,DayofMonth,6.953094e+03
2,DayOfWeek,5.446962e+02
3,Actual_Shipment_Time,3.672496e+07
4,Planned_Shipment_Time,2.025423e+07
5,Planned_Delivery_Time,1.714078e+07
6,Carrier_Num,7.941823e+05
7,Planned_TimeofTravel,8.771008e+04
8,Shipment_Delay,1.094900e+08
9,Distance,5.431833e+05


In [9]:
#checking for highy correlated feature 
threshold=0.8
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

correlation(fed_new.iloc[:,:],threshold)

{'Distance', 'Planned_Shipment_Time'}

In [11]:
fed_updated=fed.drop(['Year','Month','DayofMonth','Source','Destination','Carrier_Name','Planned_Shipment_Time'],axis=1)

#Creating dummy for day of week

day_dummy=pd.get_dummies(fed_updated['DayOfWeek'])

#renaming all dummyies column

day_dummy.columns=["DayOfWeek"+str(i) for i in range(0, 7)]

#dropping Day of Week

fed_updated=fed_updated.drop(['DayOfWeek'],axis=1)

#creating frame fo cocating dummy and data set

frames=[fed_updated,day_dummy]

In [12]:
#concating fed_updated and day dummy
fed_final=pd.concat(frames,axis=1)
#checking info 

fed_final.info()
fed_final.head()
fed_final.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3522167 entries, 0 to 3604174
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Actual_Shipment_Time   float64
 1   Planned_Delivery_Time  int64  
 2   Carrier_Num            int64  
 3   Planned_TimeofTravel   float64
 4   Shipment_Delay         float64
 5   Distance               int64  
 6   Delivery_Status        float64
 7   DayOfWeek0             uint8  
 8   DayOfWeek1             uint8  
 9   DayOfWeek2             uint8  
 10  DayOfWeek3             uint8  
 11  DayOfWeek4             uint8  
 12  DayOfWeek5             uint8  
 13  DayOfWeek6             uint8  
dtypes: float64(4), int64(3), uint8(7)
memory usage: 238.5 MB


,Actual_Shipment_Time,Planned_Delivery_Time,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Distance,Delivery_Status,DayOfWeek0,DayOfWeek1,DayOfWeek2,DayOfWeek3,DayOfWeek4,DayOfWeek5,DayOfWeek6
3604170,1059.0,1256,1547,236.0,-1.0,1572,0.0,0,0,0,1,0,0,0
3604171,555.0,738,1548,98.0,-5.0,515,0.0,0,0,0,1,0,0,0
3604172,821.0,1003,1548,96.0,-6.0,432,0.0,0,0,0,1,0,0,0
3604173,718.0,852,1551,77.0,-17.0,191,0.0,0,0,0,1,0,0,0
3604174,1127.0,1130,1551,159.0,96.0,906,1.0,0,0,0,1,0,0,0


In [13]:
# Normalization function 
def norm_func(i):
    x = (i-i.min())	/ (i.max()-i.min())
    return (x)

# Normalized data frame (considering the numerical part of data)
df_norm = norm_func(fed_final)
df_norm.describe()

,Actual_Shipment_Time,Planned_Delivery_Time,Carrier_Num,Planned_TimeofTravel,Shipment_Delay,Distance,Delivery_Status,DayOfWeek0,DayOfWeek1,DayOfWeek2,DayOfWeek3,DayOfWeek4,DayOfWeek5,DayOfWeek6
count,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06,3.522167e+06
mean,5.575077e-01,6.333849e-01,2.265182e-01,2.255559e-01,4.031192e-02,1.455495e-01,2.038785e-01,1.477945e-01,1.437334e-01,1.468295e-01,1.490943e-01,1.483902e-01,1.241225e-01,1.400357e-01
std,2.006682e-01,2.058692e-01,1.988732e-01,1.018153e-01,1.414173e-02,1.137221e-01,4.028797e-01,3.548962e-01,3.508192e-01,3.539359e-01,3.561814e-01,3.554864e-01,3.297212e-01,3.470241e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.868278e-01,4.722340e-01,6.263477e-02,1.532847e-01,3.438843e-02,6.402747e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.535640e-01,6.430691e-01,1.602834e-01,1.985401e-01,3.556077e-02,1.159362e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,7.219675e-01,8.096651e-01,3.667728e-01,2.700730e-01,3.985932e-02,1.922844e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [14]:
#separating i/p and output

x=df_norm.iloc[:,[0,1,2,3,4,5,7,8,9,10,11,12,13]]
y=df_norm['Delivery_Status']

In [15]:
#converting into float

y=np.asarray(y).astype('float32')

from sklearn.model_selection import train_test_split

x_train, x_test, y_train ,y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [16]:
model=Sequential()
model.add(Dense(x.shape[1],activation='relu',input_dim=x.shape[1]))
model.add(Dense(150,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=50000)
#evaluating model

test_mse_score, test_mae_score = model.evaluate(x_test, y_test)

Epoch 1/20
57/57 [==============================] - 5s 12ms/step - loss: 0.5421 - accuracy: 0.7961
Epoch 2/20
57/57 [==============================] - 1s 10ms/step - loss: 0.5043 - accuracy: 0.7961
Epoch 3/20
57/57 [==============================] - 1s 10ms/step - loss: 0.4799 - accuracy: 0.7961
Epoch 4/20
57/57 [==============================] - 1s 10ms/step - loss: 0.4661 - accuracy: 0.7970
Epoch 5/20
57/57 [==============================] - 1s 10ms/step - loss: 0.4538 - accuracy: 0.8018
Epoch 6/20
57/57 [==============================] - 1s 11ms/step - loss: 0.4325 - accuracy: 0.8098
Epoch 7/20
57/57 [==============================] - 1s 11ms/step - loss: 0.3977 - accuracy: 0.8260
Epoch 8/20
57/57 [==============================] - 1s 10ms/step - loss: 0.3535 - accuracy: 0.8488
Epoch 9/20
57/57 [==============================] - 1s 11ms/step - loss: 0.3055 - accuracy: 0.8731
Epoch 10/20
57/57 [==============================] - 1s 10ms/step - loss: 0.2623 - accuracy: 0.8935
Epoch 11/

In [18]:
#prediction
predict=model.predict(x_test)
error=np.mean(y_test)-np.mean(predict)
